In [1]:
#Kompresja LZW

#jednocześnie tworzenie słownika i kodowanie

In [176]:
from random import randint
import random as random
from math import log
import bitarray as ba
import operator
import struct

In [177]:
def read_file(file):
    return open(file).read()

In [178]:
#słownik dla liczb od 0 - 255
def generateDictionary():
    dict = {}
    for i in range(256):
        dict[bin(i)[2:].zfill(8)] = i
    
    return dict

In [222]:
def save_file(file_name, code):
    file = open(file_name, 'w')
    
    file.write(str(code))
    
    file.close()
    return file

In [179]:
def read_file_bin(file_name):
    bit_array = ba.bitarray()
    with open(file_name, 'rb') as fh:
        bit_array.fromfile(fh)
    return bit_array

In [180]:
def save_file_bin(file_name, bit_array):
    with open(file_name, 'wb') as fh:
        bit_array.tofile(fh)
    return 1

In [223]:
#Zapis kodu i zakodowanego tekstu do pliku
def save(file_name, file, code_len):
    
    save_file_bin(file_name, file)  
    save_file("kod", code_len)
    
    return 0

In [201]:
#Koduje tekst
def encode(text, maxDictionarySize):
    dictionary = generateDictionary()
    
    numbers = []
    max_dict_val = max(dictionary.values())
    
    
    coded_text = ba.bitarray()
    
    p = text[0:8].to01()
    
    for i in range(8, len(text), 8):
        #c - obecny ciąg binarny
        c = text[i:i+8].to01()
        
        if(len(dictionary) < maxDictionarySize or maxDictionarySize == -1):
            #słownik nie osiągnąl maksymalnego rozmiaru
            if p + c in dictionary:
                #kod znajduje się w słowniku
                p = p + c
            else:
                #kod nie znajduje się w słowniku
                numbers.append(dictionary[p])
                max_dict_val += 1
                dictionary[p + c] = max_dict_val                        
                p = c
        else:        
            if p + c in dictionary:
                #kod znajduje się w słowniku
                p = p + c
            else:
                #kod nie znajduje się w słowniku
                numbers.append(dictionary[p])                   
                p = c
    if(p in dictionary):    
        numbers.append(dictionary[p])
    else:
        numbers.append(dictionary[c])
        
    max_number = max(numbers)
    max_length = len(bin(max_number)[2:])
    
    

    for number in numbers:
        for b in bin(number)[2:].zfill(max_length):
            if b == '1':
                coded_text.append(True)
            else:
                coded_text.append(False) 
  
    print("done")
    return coded_text, max_length


In [196]:
#Dekoduje tekst
def decode(coded_text, code_len):
    
    dictionary = generateDictionary()
    
    #swap key and value
    dictionary = dict((v,k) for k,v in dictionary.items())
    
    max_dict_val = max(dictionary.keys())
    
    encoded_text = ba.bitarray()    
    
    #pierwszy element dodany do słownika
    p = coded_text[0:code_len].to01()
    p = int(p, 2)
    max_dict_val += 1 
    dictionary[max_dict_val] = dictionary[p]
    
    for b in dictionary[p]:      
        if b == '1':
            encoded_text.append(True)
        else:
            encoded_text.append(False)    

    #ucięcie nadmiarowych bitów jeżeli wystąpiły podczas zapisywania
    length = (len(coded_text) // code_len) * code_len

    for i in range(code_len, length, code_len):
        #c - obecny ciąg binarny
        c = coded_text[i:i+code_len].to01()
        c = int(c, 2) 

        dictionary[max_dict_val] = dictionary[max_dict_val] + dictionary[c][:8]
        
        for b in dictionary[c]:
            if b == '1':
                encoded_text.append(True)
            else:
                encoded_text.append(False) 
        
        max_dict_val += 1 
        dictionary[max_dict_val] = dictionary[c]
        
    print("done")
    return encoded_text


In [230]:
#Wczytuje plik binarny
def load(file_name):
    
    file = read_file_bin(file_name)
    code_len = read_file("kod")
    return file, int(code_len)

In [198]:
def compare_strings(original, new_string):
    for org, new in zip(original, new_string):
        if org != new:
            print(org, "!=", new)
            return "Teksty są różne"
    return "Teksty sa równe"

In [199]:
#Odczytanie oryginalnego pliku
wikipedia = read_file_bin("norm_wiki_sample.txt")

In [202]:
#Zakodowanie pliku
coded_wikipedia, code_len = encode(wikipedia, pow(2, 18))

done


In [224]:
#Zapisanie zakodowanego pliku
save("coded_wikipedia", coded_wikipedia, code_len)

0

In [231]:
#Wczytanie zakodowanego pliku
coded_wikipedia, code_len = load("coded_wikipedia")

In [233]:
#Odkodowanie pliku
decoded_wikipedia = decode(coded_wikipedia, code_len)

done


In [206]:
#Zapisanie odkodowanego pliku
save("wikipedia2.txt", decoded_wikipedia)

0

In [207]:
#Porównanie plików bit po bicie
compare_strings(decoded_wikipedia, wikipedia)

'Teksty sa równe'

In [208]:
1 - len(coded_wikipedia) / len(wikipedia)

0.6170131989784725

In [209]:
1 - len(wikipedia) / len(coded_wikipedia)

-1.6110560398758769

In [210]:
#Wyniki
#norm_wiki_sample.txt
#bez ograniczeń
    #Zmniejszenie rozmiaru o 61%
#2^18
    #Zmniejszenie rozmiaru o 61%
#2^12
    #Zmniejszenie rozmiaru o 49%

#wiki_sample.txt
#bez ograniczeń
    #Zmniejszenie rozmiaru o 60%
#2^18
    #Zmniejszenie rozmiaru o 59%
#2^12
    #Zmniejszenie rozmiaru o 44%

#lena.bmp
#bez ograniczeń
    #Zmniejszenie rozmiaru o 35%
#2^18
    #Zwiększenie rozmiaru o 14%
#2^12
    #Zmniejszenie rozmiaru o 10%
